In [1]:
import pandas as pd
from pathlib import Path

In [2]:
root = Path('../output/runs')

In [3]:
def classes(run):
    csv_path = str(root/run/Path('predict_test.csv'))
    df = pd.read_csv(csv_path)
    pred_counts = df['camera'].value_counts()
    pred_counts /= 0.01 * df.shape[0]
    return pred_counts

In [5]:
classes('resnet50_input_224')

Motorola-Nexus-6        13.939394
Motorola-Droid-Maxx     13.863636
Motorola-X              12.613636
iPhone-6                10.265152
Samsung-Galaxy-Note3     9.469697
HTC-1-M7                 9.318182
Samsung-Galaxy-S4        8.825758
LG-Nexus-5x              8.636364
Sony-NEX-7               7.310606
iPhone-4s                5.757576
Name: camera, dtype: float64

In [6]:
classes('resnet50_input_224_cls_lr_1e-6') # 

Motorola-Droid-Maxx     14.280303
Motorola-Nexus-6        14.090909
Motorola-X              11.931818
HTC-1-M7                 9.962121
iPhone-6                 9.507576
LG-Nexus-5x              9.204545
Samsung-Galaxy-S4        9.090909
Samsung-Galaxy-Note3     9.015152
Sony-NEX-7               8.750000
iPhone-4s                4.166667
Name: camera, dtype: float64

In [7]:
classes('resnet50_+flickr_train_lr_1e-5_pat_6')

Motorola-Droid-Maxx     11.742424
Motorola-Nexus-6        11.704545
Motorola-X              10.946970
HTC-1-M7                10.227273
Samsung-Galaxy-S4       10.000000
Samsung-Galaxy-Note3     9.810606
Sony-NEX-7               9.696970
iPhone-6                 9.431818
iPhone-4s                8.371212
LG-Nexus-5x              8.068182
Name: camera, dtype: float64

In [8]:
classes('resnet50_+flickr_train_lr_1e-5')

Motorola-Droid-Maxx     11.893939
Motorola-Nexus-6        10.946970
iPhone-6                10.606061
Samsung-Galaxy-S4       10.416667
Motorola-X              10.151515
Sony-NEX-7              10.113636
HTC-1-M7                10.037879
iPhone-4s                9.053030
Samsung-Galaxy-Note3     8.787879
LG-Nexus-5x              7.992424
Name: camera, dtype: float64

In [9]:
classes('resnet50_no_fc_train')

Motorola-Droid-Maxx     11.780303
Motorola-Nexus-6        11.590909
Motorola-X              11.098485
HTC-1-M7                10.719697
Samsung-Galaxy-S4       10.265152
iPhone-6                10.113636
iPhone-4s                9.280303
Sony-NEX-7               8.750000
Samsung-Galaxy-Note3     8.750000
LG-Nexus-5x              7.651515
Name: camera, dtype: float64

In [13]:
classes('resnet50_final_conv_rand_3')

Motorola-Droid-Maxx     12.310606
Motorola-Nexus-6        11.212121
Samsung-Galaxy-S4       10.757576
Motorola-X              10.151515
Sony-NEX-7              10.075758
iPhone-6                 9.696970
iPhone-4s                9.621212
Samsung-Galaxy-Note3     9.583333
HTC-1-M7                 9.280303
LG-Nexus-5x              7.310606
Name: camera, dtype: float64

In [14]:
classes('resnet50_repeat_best')

Motorola-Droid-Maxx     12.007576
Motorola-Nexus-6        11.439394
Samsung-Galaxy-S4       10.681818
Motorola-X              10.151515
Samsung-Galaxy-Note3    10.037879
iPhone-6                 9.924242
Sony-NEX-7               9.507576
HTC-1-M7                 9.280303
iPhone-4s                9.242424
LG-Nexus-5x              7.727273
Name: camera, dtype: float64

In [15]:
classes('resnet50_add_note3')

Motorola-Droid-Maxx     13.333333
Motorola-Nexus-6        12.234848
Samsung-Galaxy-S4       10.189394
Sony-NEX-7              10.000000
Motorola-X               9.886364
iPhone-6                 9.886364
Samsung-Galaxy-Note3     9.393939
iPhone-4s                9.166667
HTC-1-M7                 8.863636
LG-Nexus-5x              7.045455
Name: camera, dtype: float64